*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

**CD Measurements - Data Analysis**

# DNA Origami 

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
import openpyxl
import re

from scipy import integrate
import lmfit
import os

In [2]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
# change plot size
plt.rcParams["figure.figsize"] = (10,7)

"""# activate Latex, change font type
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Helvetica"],
})"""

'# activate Latex, change font type\nplt.rcParams.update({\n    "text.usetex": True,\n    "font.family": "sans-serif",\n    "font.serif": ["Helvetica"],\n})'

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [5]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"
path = "F:\\HZDR\\0_Cloud\\CD_Jasco_815\\DNA\Origami\\Triangle\\1-99_Denaturation"

# My Path:
#path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\DNA Origami"
#path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\1-99_Denaturation"
datalist = os.listdir(path)
print(datalist)

['0 M Control 10 mM Mg', '05 M Gdm2SO4 100 mM Mg', '05 M Gdm2SO4 400 mM Mg', '1 M GdmCl 100 mM Mg', '1 M GdmCl 400 mM Mg', '1 M GdmCl 400 mM Mg 2', '2 M Gdm2SO4 100 mM Mg', '2 M Gdm2SO4 400 mM Mg', '4 M GdmCl', '4 M GdmCl 100 mM Mg', '4 M GdmCl 100 mM Mg 2', '4 M GdmCl 400 mM Mg']


In [10]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [7]:
# options to acces data
print(data_all['0 M Control 10 mM Mg'])
# multiple options
print(list(map(data_all.get, ['0 M Control 10 mM Mg', '4 M GdmCl'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x000001FBE0CA0880>, <cdata.CData object at 0x000001FBE0C89C70>]
(<cdata.CData object at 0x000001FBE0CA0880>, <cdata.CData object at 0x000001FBE0BB1880>, <cdata.CData object at 0x000001FBE0CFFA00>, <cdata.CData object at 0x000001FBE0D28070>)


## Smooth Data

In [8]:
data_all_smooth = {}
for key in data_all.keys():

    
    window_size = 5
    tail = window_size // 2
    df = data_all[key].cd_df
    df_new = pd.DataFrame(index=df.index)
    
    for i in range(tail, len(df.columns[tail:-tail])+1): 
        df_new[df.columns[i]] = df.iloc[:, i]
        
        for k in range(1, tail+1):
            df_new[df.columns[i]] = df_new[df.columns[i]] + df.iloc[:, i-k] + df.iloc[:, i+k]
        
        df_new[df.columns[i]] = df_new[df.columns[i]] / window_size
    data_all_smooth[key] = df_new
    
    #if printed
    #print_path = "F:\\HZDR\\CD_data"
    #name = os.path.join(print_path, key + '_smoothed.txt')
    #df_new.to_csv(name, sep='\t', index=True)

## PCA

### Print-out for PCA


<div class="alert alert-block alert-info">
<b>Info:</b> Creates the .dat - files which are used for PCA and ITTFA
</div>

In [9]:
print(data_all['Control_0M'].t_list)

KeyError: 'Control_0M'

In [ ]:
# general path where files should be saved without last folder name

# Daniels path
print_path = "F:\\HZDR\\0_Cloud\\ITTFA_PCA\\dat"
# Christophs path
#print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [ ]:
print(data_all.keys())

In [ ]:
liste = ['Control_0M']

In [ ]:
# files get seperatedly created, folder name must exist
folder_name = 'Control_0M'
i= 1

for key in liste:
    for col in data_all[key].cd_df.columns:
        if col >= 20  and col <= 90:
            name =  print_path + '\\' + folder_name + '\\' + str(i) + '.dat'
            data_all[key].cd_df.loc[:, col].to_csv(name, sep = " ", header=False)
            i = i + 1
    print(i)

In [ ]:
# extra files if new spectra should be added

#excel path
print_path = "F:\\HZDR\\0_Cloud\\ITTFA_PCA\\dat"
#excel_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Factor_analysis\\ITTFA"
# excel file name
excel_name =  excel_path + '\\' + 'Cl_4M.xlsx'

# open and modify excel 
df_excel = pd.read_excel(excel_name)
df_excel = df_excel.drop(index = 0)
df_excel = df_excel.set_index('Wavelength')

# create subfiles for spectra
for k in df_excel.columns:
    name =  print_path + '\\' + folder_name + '\\' + str(i) + '.dat'
    df_excel.loc[:, k].to_csv(name, sep = " ", header=False)
    i = i + 1
print(i)

### Eigenvector output


In [ ]:
print_path = "F:\\HZDR\\0_Cloud\\ITTFA_PCA\\dat"
#print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [ ]:
print(data_all.keys())

In [ ]:
liste = ['Control_0M']

In [ ]:
for  key in liste:
    # create DataFrame for plot
    plot_df = pd.DataFrame(index=data_all[key].cd_df.loc[215:300].index)
    
    # number of components
    components = 3
    
    # read files 
    for i in range(components):
        
        name = 'eigenvector' + str(i+1) 
        # get data path
        data_path = os.path.join(print_path, key, name + '.dat')
        
        # open file and create data list
        file = open(data_path, "r")
        data = []
        
        # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
        list_of_lines = file.readlines()
        for k in range(len(list_of_lines)):
            data.extend(re.split(' |\n', list_of_lines[k]))
        data1 = np.array(list(filter(lambda x: x != '', data)))
        
        # data list reshaped as 2D array and added to plot
        data1 = data1.reshape(86, 2)
        plot_df[i+1] = data1[:, 1]
        
    # same procedure for eigenvalues to norm eigenvectors    
    name = 'eigenvalues'
    data_path = os.path.join(print_path, key, name + '.dat')
    # open file and create data list
    file = open(data_path, "r")
    data = []
    # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
    list_of_lines = file.readlines()
    for k in range(len(list_of_lines)):
        data.extend(re.split(' |\n', list_of_lines[k]))
        data1 = np.array(list(filter(lambda x: x != '', data)))
    
    # data list reshaped as 2D array and added to plot
    data1 = data1.reshape(components, 2)
    eigenvectors = data1
    
    # change dtype to float
    plot_df = plot_df.astype('float64')
    eigenvectors = eigenvectors.astype('float64')
    #plot_df = plot_df.multiply(eigenvectors.T[1])
    # norm between 0 - 1 if wanted
    #plot_df = plot_df.subtract(plot_df.min()).divide((plot_df.max()-plot_df.min()))
    plot.mult_func([1,2, 3, 4, 5, 6], [plot_df.T], linestyle=["solid","solid",  'dashed','dashed','dotted', 'dotted'], marker=["","","","","","",""], subtitle=[""], 
                   x_label="wavelength \ nm", y_label="eigenvectors")

### Varimax Output

In [ ]:
print_path = "F:\\HZDR\\0_Cloud\\ITTFA_PCA\\dat"
#print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [ ]:
files = os.listdir(print_path)
print(files)

In [ ]:
liste = ['Control_0M']

In [ ]:
for  key in liste:
    # create DataFrame for plot
    plot_df = pd.DataFrame(index=data_all[key].t_list[:-3])
    
    # number of components
    components = 4
    
    # read files 
    for i in range(components):
        
        name = 'VARIMAX_loading_component' + str(i+1) 
        # get data path
        data_path = os.path.join(print_path, key, name + '.dat')
        
        # open file and create data list
        file = open(data_path, "r")
        data = []
        
        # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
        list_of_lines = file.readlines()
        for k in range(len(list_of_lines)):
            data.extend(re.split(' |\n', list_of_lines[k]))
        data = np.array(list(filter(lambda x: x != '', data)))
        
        # data list reshaped as 2D array and added to plot
        data = data.reshape(len(data_all[key].t_list[:-3]), 2)
        plot_df[i+1] = data[:, 1]
   
    # change dtype to float
    plot_df = plot_df.astype('float64')
    # norm between 0 - 1 if wanted
    plot_df = plot_df.subtract(plot_df.min()).divide((plot_df.max()-plot_df.min()))
    plot.mult_func([1, 2, 3, 4], [plot_df.T], linestyle=["solid","dashdot","dashed","dotted"], marker=["o","o","o","o"], subtitle=[key], 
                   x_label="Temperature [°C]", y_label="Abstract Concentration after Varimax Rotation")

In [ ]:
print(data_all[key].t_list, melt_T[key])
print(data_all[key].t_list[2:])

### ITTFA Output

In [ ]:
print_path = "F:\\HZDR\\0_Cloud\\ITTFA_PCA\\dat"
#print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\PCA"

In [ ]:
key_list = ['Control_0M']
filename = 'Control_0M'
wave_min = 215
wave_max = 300 

t_min = 20
t_max = 90
t_list = []

for key in key_list:
    t_list.extend(data_all[key].cd_df.loc[wave_min:wave_max,t_min:t_max].columns)
    # extend for extra spectra
    #t_list.insert(0, 10)
    #t_list.extend([100, 110,120,130])

# create DataFrame for plot
conc_df = pd.DataFrame(index=t_list)
spec_df = pd.DataFrame(index=data_all[key].cd_df.loc[wave_min:wave_max,t_min:t_max].index)

# number of components
components = 3
    
# read files, 1st rel_conectration, 2nd spectra of components and third error in rel concentrations
for i in range(components):
    name1 = 'rel_concentration_component' + str(i+1) 
    name2 = 'component' + str(i+1) 
    
    # get data path
    data_path1 = os.path.join(print_path, filename, name1 + '.dat')
    data_path2 = os.path.join(print_path, filename, name2 + '.dat')
        
    # open file and create data list
    file1 = open(data_path1, "r")
    file2 = open(data_path2, "r")
    data1 = []
    data2 = []
        
    # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
    list_of_lines1 = file1.readlines()
    list_of_lines2 = file2.readlines()
    
    for k in range(len(list_of_lines1)):
        data1.extend(re.split(' |\n', list_of_lines1[k]))
    data1 = np.array(list(filter(lambda x: x != '', data1)))
    
    for j in range(len(list_of_lines2)):
        data2.extend(re.split(' |\n', list_of_lines2[j]))
    data2 = np.array(list(filter(lambda x: x != '', data2)))
    
    # data list reshaped as 2D array and added to plot
    data1 = data1.reshape(len(t_list), 2)
    conc_df[i+1] = data1[:, 1]
    
    data2 = data2.reshape(len(spec_df.index), 2)
    spec_df[i+1] = data2[:, 1]

# read relative errors of concentrations
name3 = 'error_in_rel_concentrations'
data_path3 = os.path.join(print_path, filename, name3 + '.dat')
file3 = open(data_path3, "r")
data3 = []
list_of_lines3 = file3.readlines()
for j in range(len(list_of_lines3)):
    data3.extend(re.split(' |\n', list_of_lines3[j]))
data3 = np.array(list(filter(lambda x: x != '', data3)))
data3 = data3.reshape(components, 2)



# change dtype to float
conc_df = conc_df.astype('float64')
spec_df = spec_df.astype('float64')

# trying to calculate error
data_measured = data_all[key].cd_df.loc[wave_min:wave_max,t_min:t_max]
data_calculated = pd.DataFrame(np.matmul(spec_df.to_numpy(), conc_df.to_numpy().T), index=data_measured.index, columns=t_list)

# renorm data
data_measured = data_measured.divide(data_measured.max(axis=0) - data_measured.min(axis=0))
data_calculated  = data_calculated.divide(data_calculated.max(axis=0) - data_calculated.min(axis=0))
df_error = np.sqrt(np.square(data_measured.subtract(data_calculated.loc[:,20:90])).sum(axis=0)) / (wave_max - wave_min - 1)
#print(df_error.max())
# plot.mult_func([20], [data_measured, data_calculated], swap=True)  # plot both together

# negative reverse and amplitude adaption
#conc_df[2] = conc_df[2] * -1
#spec_df[2] = spec_df[2] * -1
#conc_df = conc_df * (data_all[key].cd_df.loc[247, 20]/spec_df.loc[247,1])
#spec_df = spec_df * (data_all[key].cd_df.loc[247, 20]/spec_df.loc[247,1])
# norm between 0 - 1 if wanted
#conc_df = conc_df.subtract(conc_df.min()).divide((conc_df.max()-conc_df.min()))

plot.mult_func(np.arange(1, components + 1), [conc_df.T], linestyle=["solid","dashdot","dashed","dotted"], subtitle=[filename], 
                x_label="temperature / °C", y_label="relative population  / %", 
               label=[*np.arange(1, components + 1), "Melting temperature"], title="Concentration of Components after ITTFA",
               marker=["o", "X", "s"], 
               baseline=True, 
               x_max=75
               #vertical_line=[melt_T[filename]]
              )    
    
plot.mult_func(np.arange(1, components + 1), [spec_df.T], linestyle=["solid","dashdot","dashed","dotted"], marker=["","","",""], subtitle=[filename], 
                x_label="wavelength / nm", title="Spectra of components from ITTFA", baseline=True, label=np.arange(1, components + 1),
               y_max = [spec_df.T.loc[components].max().max()], y_min = [spec_df.T.loc[components].min().min()], y_scaling=False)    

In [ ]:
plot.mult_func([1,3,2], [spec_df.T], marker=["","","",""], subtitle=[''], 
                x_label="wavelength /nm", baseline=True, label=['S1','S2\'', 'S3'], y_label='abstract ellipticity',
              colors=['tab:red', 'tab:grey', 'tab:green'])  

Recalculation and TS-Plots

In [ ]:
df = pd.DataFrame(np.matmul(spec_df.to_numpy(), conc_df.to_numpy().T), index=data_measured.index, columns=t_list)
max_cd_value = df.abs().max().max()
sync,  assync = ana.perturbation_moving_window(df, window_size=3)
plot.heatmap(sync, x_min=[215], x_max=[300], x_label='temperature / °C', y_label='wavelength / nm',
                 c_label="",c_min=[-1e8], c_max=[1e8], title="", subtitle=[''],
                contour_lines=[-0.7*max_cd_value, -0.4*max_cd_value,  -0.2*max_cd_value,  0, 0.2*max_cd_value, 
                                0.4*max_cd_value, 0.7*max_cd_value],
            hline=melt_T[filename])

In [ ]:
print(data3, df_error)

In [ ]:
print(data_all['GdmCl_1M'].t_list)

*write numbers into excel*

In [ ]:

#print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data"

In [ ]:

excel_path = print_path + "\\" + key + ".xlsx"
#with pd.ExcelWriter(excel_path, mode='w') as writer:  
    #    data_all[key].absorb_df.to_excel(writer, sheet_name="concentration ")

In [ ]:
with pd.ExcelWriter(excel_path, mode='w') as writer:  
    conc_df.round(6).to_excel(writer, sheet_name="concentration ")
    
with pd.ExcelWriter(excel_path, mode='a') as writer:  
    spec_df.round(6).to_excel(writer, sheet_name="spectra")
            

In [ ]:
plot.mult_func([2, 90], [data_measured.T, data_calculated.T], subtitle=['calculated vs measured spectra of 2nd component'], label=['measured', 'calculated'], x_label='Wavelength / nm')

## Plots

### Heat Maps

In [11]:
#liste =  ["Control_0M", 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M', 'GdmCl_6M', 'Gdm2SO4_0.5M','Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M']
liste = ['0 M Control', '05 M Gdm2SO4 100 mM Mg', '05 M Gdm2SO4 400 mM Mg', '1 M GdmCl 100 mM Mg', '1 M GdmCl 400 mM Mg', '1 M GdmCl 400 mM Mg 2', '2 M Gdm2SO4 100 mM Mg', '2 M Gdm2SO4 400 mM Mg', '4 M GdmCl', '4 M GdmCl 100 mM Mg', '4 M GdmCl 100 mM Mg 2', '4 M GdmCl 400 mM Mg']

In [12]:
for key in data_all.keys():
    df = data_all[key].cd_df.loc[215:300]
    max_cd_value = df.abs().max().max()
    plot.heatmap(df, subtitle=[key], title="", x_min=[215], x_max=[300], swap=True,
                 x_label=r"temperature $ / \ ^{\circ}C$", y_label=r"wavelength $/ \ nm$", c_label=r"molar ellipticity "
                 r"$ / \ \frac{deg \times cm^{2}}{dmol}$" , 
                 contour_lines=[-0.8*max_cd_value, -0.5*max_cd_value,  -0.25*max_cd_value,  0, 0.25*max_cd_value, 0.5*max_cd_value, 0.8*max_cd_value]
                )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
for key in data_all_smooth.keys():
    df = data_all_smooth[key].loc[215:300]
    max_cd_value = df.abs().max().max()
    plot.heatmap(df, subtitle=[key], title="", x_min=[215], x_max=[300], swap=True,
                 x_label=r"temperature $ / \ ^{\circ}C$", y_label=r"wavelength $/ \ nm$", c_label=r"molar ellipticity "
                 r"$ / \ \frac{deg \times cm^{2}}{dmol}$" 
                )

In [ ]:
for key in data_all.keys():
    plot.heatmap(data_all[key].cd_df, subtitle=[""], title=str(key), x_min=[215], swap=True, y_label="Temperature [°C]", x_label="Wavelength [nm]")

In [ ]:
#plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values", x_min=[220]*len(data_all), swap=False)

### 3D surface plots

In [ ]:
for key in data_all.keys():
    df = data_all[key].cd_df.loc[215:310, 1:80]

    plot.function3d(df, title="", z_label=r"molar CD \ "
                 r"$ \frac{deg \times cm^{2}}{dmol}$",
               x_label='wavelength \ nm', y_label='temperature \ °C')

###  Function Plots 

#### CD-Spectra Plot

In [ ]:
df =  data_all["0 M Control 10 mM Mg"].cd_df

plot.mult_func([20], [df], swap=True, x_min=215, x_max=300, marker=[''], subtitle=[''], baseline=True, colors=['tab:red'], x_label="wavelength / nm", label=['20 °C'])

In [ ]:
wave_min = 215
wave_max = 330

plot.mult_func([230, 247, 275], [data_all['Control_0M'].cd_df.loc[wave_min:wave_max]], 
               swap=False, x_label="Temperature [°C]", subtitle=[""], 
               marker=['', '', '', '', '', ''], linestyle=['-', ':', ':', '', '-'], baseline=True, 
               label=['230','247','275'])

In [ ]:
wave_min = 215
wave_max = 300

key_list = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M' ,'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
cd_liste = []
for key in key_list:
    cd_liste.append(data_all[key].cd_df)
    
plot.mult_func([20], cd_liste, title=key, swap=True, x_min=wave_min, x_max=wave_max,
                   x_label="Wavelength [nm]", y_label="CD values [mdeg]", subtitle=[""],  baseline=True, label=key_list,
               marker=[""]*len(key_list)
                  )



In [ ]:
liste = np.zeros(len(key_list))
for i in range(len(key_list)):
    liste[i] = cd_liste[i].loc[220,20]
print(np.mean(liste), np.std(liste), np.mean(liste)/np.std(liste))

#### Max/Min Plots


multiple probes

In [ ]:
print(data_all[key])

In [ ]:
# Daniels path:
excel_path = "F:\\HZDR\\CD_Auswertung\\Exel_Max-Min"
# Christophs path:
#excel_path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\output.xlsx"

In [ ]:
# liste 
#liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M',  'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h' , 'GdmCl_4M']
liste = ['05 M Gdm2SO4 100 mM Mg', '05 M Gdm2SO4 400 mM Mg', '1 M GdmCl 100 mM Mg', '1 M GdmCl 400 mM Mg', '2 M Gdm2SO4 100 mM Mg', '2 M Gdm2SO4 400 mM Mg', '4 M GdmCl', '4 M GdmCl 100 mM Mg', '4 M GdmCl 100 mM Mg 2', '4 M GdmCl 400 mM Mg']
wave_min = 260
wave_max =  280

for key in data_all.keys():
    plot_data =  ana.min_wave(data_all[key].cd_df.abs(), wave_min=wave_min, wave_max=wave_max)
    plot_data = plot_data.drop(['Value'], axis=1)   
    plot_data.rename(columns = {'Wavelength': key}, inplace = True)
    plot.mult_func([key], [plot_data.T], y_scaling=True, y_min=[260], y_max=[265],)
    #with pd.ExcelWriter(excel_path, mode='a') as writer:  
                    #plot_data.to_excel(writer, sheet_name=key)

In [ ]:
wave_min = 260
wave_max =  280
key = "4 M GdmCl 400 mM Mg"

# First data you want to add to the plot
plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
plot_data = plot_data.drop(['Wavelength'], axis=1)   
plot_data.rename(columns = {'Value': key}, inplace = True)



# other Data you want to add, if always same max/min values then loop would be possible
#plot_data["GdmCl_4M"] = ana.max_wave(data_all["GdmCl_4M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["Gdm2SO4_2M"] = ana.max_wave(data_all["Gdm2SO4_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["GdmSCN_2M"] 2= ana.max_wave(data_all["GdmSCN_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]


plot.mult_func([key], [plot_data.T], subtitle=[""], title="Max. CD Value between " + str(wave_min) + ' nm - ' +str(wave_max) + " nm")

one probe

In [ ]:
wave_min = 260
wave_max =  300
title = "Max. CD-Value between " + str(wave_min) + "nm - " + str(wave_max) +"nm"

for key in data_all.keys():
    plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
    plot.mult_func(["Value"], [plot_data.T], subtitle=[""],  title=title, label=[key, ])



## Correlation Analyis

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2007). Two-dimensional correlation analysis useful for spectroscopy, chromatography, and other analytical measurements. Analytical Sciences, 23(2), 139–146. https://doi.org/10.2116/analsci.23.139
</div>

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2015). Techniques of two-dimensional (2D) correlation spectroscopy useful in life science research. Biomedical Spectroscopy and Imaging, 4(2), 109–127. https://doi.org/10.3233/bsi-150105
</div>

### Homogenous Spectrum

*optional: pareto scaling, auto scaling and Reference and projection*

In [ ]:
# list of measurements on which correlation should be performed
liste =  ['4 M GdmCl 400 mM Mg']
#'Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M'
# min and max temperature between which correlations should be analysed
t_min = 1
t_max = 80

for key in liste:
    
    df =  data_all[key].cd_df.loc[215:300, t_min:t_max]
    # get reference
    ref = data_all[key].cd_df.loc[215:300, 20]

    sync, assync =  ana.correlation(df, center=True)

    plot.heatmap(sync,  title="", c_label= "", subtitle=["Synchronous Spectrum"], y_label="wavelength / nm", x_label="wavelength / nm", c_min=[], c_max=[])
    plot.heatmap(assync,  title="", c_label= "", subtitle=["Assynchronous Spectrum"], y_label="wavelength / nm", x_label="wavelength / nm", c_min=[], c_max=[])

In [ ]:
df = data_all["4 M GdmCl 400 mM Mg"].cd_df.loc[220:300]


# getting index and shared column values
idx = df.index
col = df.columns

# extend measurement points and to create delta_t
t_list_extended = np.array(col)
t_list_extended = np.insert(t_list_extended, 0, 2 * col[0] - col[1])
t_list_extended = np.append(t_list_extended, 2 * col[-1] - col[-2])
delta_t = t_list_extended[2:] - t_list_extended[:-2]

# from average spectrum
dyn = ana.centering(df)

# transform to numpy array
dyn = dyn.to_numpy()
rows, cols = dyn.shape

# creating Hilbert-Noda-matrix for async spectrum
arr = np.array(col).reshape(1, cols) - np.array(col).reshape(cols, 1) + np.identity(cols)
h_n_m = 1 / (2 * np.pi * arr)
h_n_m = np.multiply(h_n_m, delta_t)
h_n_m = (h_n_m - h_n_m.T) / 2  # change diag to zero

# calculate synchronous and asynchronous spectrum with matrix
sync_uneven = np.dot(np.multiply(dyn, delta_t), dyn.T) / (2 * (col[-1] - col[0]))

assync_uneven = np.zeros((rows, rows))

for i in range(rows):
    for j in range(rows):
                         
                         hilbert_transform = np.dot(h_n_m, dyn[j])
                         assync_uneven[i,j] = np.sum(np.multiply(dyn[i], hilbert_transform))
                    
assync_uneven = np.divide(assync_uneven, (2 * (col[-1] - col[0])))
        
#assync_uneven = np.dot(np.multiply(dyn, delta_t), np.dot(h_n_m, dyn.T)) / (2 * (col[-1] - col[0]))

sync_uneven = pd.DataFrame(sync_uneven, index=idx, columns=idx).astype('float64')
assync_uneven = pd.DataFrame(assync_uneven, index=idx, columns=idx).astype('float64')

df2 = pd.DataFrame(np.dot(h_n_m, dyn.T))
df2 = df2.astype('float64')
plot.heatmap(sync_uneven,  title="", c_label= "", subtitle=["Synchronous Spectrum"], y_label="wavelength / nm", x_label="wavelength / nm" , 
             c_min=[], c_max=[], contour_lines=[-1.5e7, -0.5e7, 0 , 0.5e7, 1.5e7])
plot.heatmap(assync_uneven,  title="", c_label= "", subtitle=["Assynchronous Spectrum"], y_label="wavelength / nm", x_label="wavelength / nm", 
             c_min=[], c_max=[], contour_lines=[-3e5, -1e5, 0, 1e5, 3e5])

In [ ]:
# list of measurements on which correlation should be performed
#liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M','GdmCl_1M', 'GdmCl_2M' , 'GdmCl_4M', 'GdmCl_6M']
#liste = ['0 M Control 10 mM Mg', '05 M Gdm2SO4 100 mM Mg', '05 M Gdm2SO4 400 mM Mg', '1 M GdmCl 100 mM Mg', '1 M GdmCl 400 mM Mg', '1 M GdmCl 400 mM Mg 2', '2 M Gdm2SO4 100 mM Mg', '2 M Gdm2SO4 400 mM Mg', '4 M GdmCl', '4 M GdmCl 100 mM Mg', '4 M GdmCl 100 mM Mg 2', '4 M GdmCl 400 mM Mg']
# min and max temperature between which correlations should be analysed
t_min = 20
t_max = 90

# collection of sync and assync plots
sync_dic = {}
async_dic ={}

# max scaling values for upscaling correlation values with absorption spectrum
max_scaling_value_sync = 0
max_scaling_value_async = 0

In [ ]:
# define witdh of additionaly absorbtion correlation band
width = 5

for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[215:300, 20]
    
    # sort out values with HT above 900
    df = data_all[key].cd_df.loc[215:300, t_min:t_max]
    
    # add absorbance as extra row 
    for i in range(width):
        df = df.append(data_all[key].absorb_df.loc[260, t_min:t_max], ignore_index = True)
        ref[301+i] = data_all[key].absorb_df.loc[260, 20]
    ref.index = np.arange(86 + width)
    
    # calculate
    sync, assync = ana.correlation(df, ref_spec=[ref], center=False)
    sync1, assync1 = ana.correlation(data_all[key].cd_df.loc[215:300, t_min:t_max], ref_spec=[data_all[key].cd_df.loc[215:300, 20]], scaling='pareto', center=False)
        
    # pareto scaling for CD-part
    sync.loc[:85, :85] = sync1.to_numpy()
    assync.loc[:85, :85] = assync1.to_numpy()
    
    # get the sacling the absorption values for sync, and assync and compare with max scaling value so far...
    max_cd_value_sync = sync.abs().max().max()
    max_abs_value_sync = sync.loc[86:].abs().max().max()
    max_scaling_value_sync = (max_cd_value_sync / max_abs_value_sync)
        
    max_cd_value_async = assync.abs().max().max()
    max_abs_value_async = assync.loc[86:].abs().max().max()
    max_scaling_value_async = (max_cd_value_async / max_abs_value_async)
        
    print(max_scaling_value_sync, max_scaling_value_async)
    # adding DataFrame to plot dics
    sync_dic[key] = sync
    async_dic[key] =  assync

In [ ]:
# max scaling values for upscaling correlation values with absorption spectrum
max_scaling_value_sync = 21.655153097330512
max_scaling_value_async = 19.74706886424258

In [ ]:
for key in sync_dic.keys():
    max_cd_value = sync_dic[key].abs().max().max()
    sync_dic[key].loc[86:] =  sync_dic[key].loc[86:] * max_scaling_value_sync
    sync_dic[key].loc[:85,86:] = sync_dic[key].loc[:85,86:] * max_scaling_value_sync
    plot.heatmap(sync_dic[key],  title='', subtitle=["" ], y_label="Wavelength 1 [nm]", 
                 x_label="Wavelength 2 [nm]", c_label="", c_min=[-max_cd_value], c_max=[max_cd_value], hline=85, vline=85,
                 contour_lines=[-0.8*max_cd_value,-0.5*max_cd_value, -0.3*max_cd_value, -0.15*max_cd_value, 0, 0.15*max_cd_value, 0.3*max_cd_value, 0.5*max_cd_value, 0.8*max_cd_value])
    
    max_cd_value = async_dic[key].abs().max().max()
    async_dic[key].loc[86:] =  async_dic[key].loc[86:] * max_scaling_value_async
    async_dic[key].loc[:85,86:] = async_dic[key].loc[:85,86:] * max_scaling_value_async
    plot.heatmap( async_dic[key], title="", subtitle=[ "" ], y_label="Wavelength 1 [nm]", 
                 x_label="Wavelength 2 [nm]",  c_label="", hline=85, vline=85,
                 contour_lines=[-0.8*max_cd_value, -0.5*max_cd_value,  -0.3*max_cd_value, -0.15*max_cd_value, 0, 0.15*max_cd_value, 0.3*max_cd_value, 
                                0.5*max_cd_value, 0.8*max_cd_value] )

### Tools for Analysis

<div class="alert alert-block alert-info">
<b>Info:</b> Change 'key' to wanted measurement!
</div>

In [ ]:
key1 = "GdmCl_2M"
key2 = "GdmCl_2M_24h"
ref1 = data_all[key1].cd_df.loc[:, 20]
ref2 = data_all[key2].cd_df.loc[:, 20]
sync, assync = ana.correlation(data_all[key1].cd_df.loc[:, :45], data_all[key2].cd_df.loc[:, :45], scaling='pareto', ref_spec=[ref1, ref2], center=False)

In [ ]:
plot.heatmap(sync, assync,  title=str(key1) + str(key2), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210], y_min=[210, 210],
                y_label= str(key1) + " WL[nm]", x_label= str(key2) + " WL[nm]")

**Diagonal Analysis**

In [ ]:
arr = sync.to_numpy()
diag =  pd.DataFrame(arr.diagonal(), index=sync.index)
wave =  list(sync.index)[:-1]
diff =  pd.DataFrame(np.diff(arr.diagonal()), index=wave)
plot.mult_func([0], [diff.T, diag.T], baseline=True, x_label="wavelength [nm]", subtitle=["Diagonal with differential"], label=["CD Values", "Differential"], x_min=210)

**Line Analyis**

In [ ]:
wavelength=247
# adjust wavelength to shift line analysis*

In [ ]:
arr = sync.loc[wavelength, :].to_numpy()
df = pd.DataFrame(arr, index=sync.index).T
diff = pd.DataFrame(np.diff(arr), index=list(sync.index)[:-1]).T
plot.mult_func([0], [df, diff], x_label="wavelength [nm]", baseline=True, x_min=210, subtitle=["Synchronous Line Analysis"], label=[wavelength, "derivative"])

**Excel**

In [ ]:
#Daniels path:
excel_path = "F:\\HZDR\\CD_Auswertung\\2Dcorr"
#Christophs path:
#excel_path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\output.xlsx"

create new document

In [ ]:
with pd.ExcelWriter(excel_path, mode='w') as writer:  

                    comp2_conc_df.round(2).to_excel(writer, sheet_name="output ")

Homgeneous Spectra Values

In [ ]:
# liste 
liste =  ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h' , 'GdmCl_4M', 'GdmCl_6M', 'Urea_2M']
t_min = 20
t_max = 90



for key in liste:
    # get reference
    ref = data_all[key].cd_df.loc[:, 20]
    # sort out values with HT above 900
    df = data_all[key].cd_df
    # calculate
    sync1, assync1 = ana.correlation(df.loc[:,:45], scaling='pareto', ref_spec=[ref], center=False)
    df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
    df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
    with pd.ExcelWriter(excel_path, mode='a') as writer:  
                    df1.to_excel(writer, sheet_name=key  + " sync")
                    df2.to_excel(writer, sheet_name=key  + " async")

Heterogeneous Spectra Values

In [ ]:
# Liste

#plot_list = ["Control_0M", 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
plot_list = ["Control_0M", 'Gdm2SO4_0.5M', 'Gdm2SO4_2M',  'Gdm2SO4_4M', 'Gdm2SO4_6M']
anzahl = len(plot_list)
t_min = 20 
t_max = 90


# calculation 
for i in range(anzahl):
    for j in range(anzahl):
        if i >= j:
            continue
        df1 = data_all[plot_list[i]].cd_df
        df2 = data_all[plot_list[j]].cd_df
        ref1 = df1.loc[:, 20]
        ref2 = df2.loc[:, 20]
        sync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling='pareto', ref_spec=[ref1, ref2], center=False)
        df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
        df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
        with pd.ExcelWriter(excel_path, mode='a') as writer:  
            df1.to_excel(writer, sheet_name=plot_list[i] + " vs.  " + plot_list[j]  + " sync")
            df2.to_excel(writer, sheet_name=plot_list[i] + " vs.  " + plot_list[j]  + " async")
            
# extra calculation for 2M vs 2M_24h            
"""df1 = data_all["GdmCl_2M"].cd_df
df2 = data_all["GdmCl_2M_24h"].cd_df
ref1 = df1.loc[:, 20]
ref2 = df2.loc[:, 20]
sync1, assync1 = ana.correlation(df1.loc[:, :t_max], df2.loc[:, :t_max], scaling='pareto', ref_spec=[ref1, ref2], center=False)
df1 = sync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
df2 = assync1.loc[[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312],[212, 215, 218, 221, 223, 229, 247, 261, 265, 268, 274, 281, 284, 294, 305, 312]].round(3)
with pd.ExcelWriter(excel_path, mode='a') as writer:  
    df1.to_excel(writer, sheet_name="2M vs. 2M 24h sync")
    df2.to_excel(writer, sheet_name="2M vs. 2M 24h async")"""

In [ ]:
arr = assync.loc[wavelength, :].to_numpy()
df = pd.DataFrame(arr, index=assync.index).T
diff = pd.DataFrame(np.diff(arr), index=list(assync.index)[:-1]).T
plot.mult_func([0], [df, diff], x_label="wavelength [nm]", baseline=True, x_min=210, subtitle=["synchronous Line Analysis"])

### Heterogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [ ]:
print(data_all.keys())

In [ ]:
#plot_list = ["Control_0M", 'GdmCl_0.5M', 'GdmCl_2M',  'GdmCl_4M', 'GdmCl_6M']
plot_list = ["Control_0M", 'Gdm2SO4_0.5M', 'Gdm2SO4_2M',  'Gdm2SO4_4M', 'Gdm2SO4_6M']
anzahl = len(plot_list)
t_min = 20 
t_max = 90

In [ ]:
for i in range(anzahl):
    for j in range(anzahl):
        if i >= j:
            continue
        title = "Heterogenoues 2D Correlation between " + plot_list[i] + " " + plot_list[j]
        df1 = data_all[plot_list[i]].cd_df
        df2 = data_all[plot_list[j]].cd_df
        ref1 = df1.loc[:, 20]
        ref2 = df2.loc[:, 20]
        sync1, assync1 = ana.correlation(df1, df2, scaling='pareto', ref_spec=[ref1, ref2], center=False)
        sync2, assync2 = ana.correlation(df1, df2, scaling='auto', ref_spec=[ref1, ref2], center=False)
        plot.heatmap(sync1, assync1, title=title, subtitle=["Synchronous Spectrum", "Asynchronous Spectrum", "Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210, 210, 210], y_min=[210, 210, 210, 210],
                y_label= plot_list[i] + " WL[nm]", x_label= plot_list[j] + " WL[nm]")

### Perturbation-Correlation Moving-Window 2D Correlation Spectroscopy

*different window size can be choosen*

In [ ]:
liste = ['1_M_GdmCl_100_mM_Mg',  '4_M_GdmCl_100_mM_Mg', '4_M_GdmCl_10_mM_Mg', '4_M_GdmCl_400_mM_Mg']
liste = 

In [ ]:
for key in liste:
    df = data_all[key].cd_df.loc[230:300]
    max_cd_value = df.abs().max().max()
    sync, assync, = ana.perturbation_moving_window(df, window_size=10)
    plot.heatmap(sync, x_min=[230], x_max=[300], 
                 c_label="", title="", subtitle=[key],
                 c_max = [0.5e7], c_min=[-0.5e7],
                #contour_lines=[-0.7*max_cd_value, -0.4*max_cd_value,  -0.2*max_cd_value,  0, 0.2*max_cd_value,  0.4*max_cd_value, 0.7*max_cd_value],
                hline=melt_T[key])

## Absorbance Values

In [ ]:
for key in data_all.keys():
    plot.heatmap(data_all[key].absorb_df, subtitle=[key], title="Absorbance Values", x_min=[215], x_max=[300], c_min=[0]*len(data_all))


In [ ]:
for key in data_all.keys():
    df = data_all[key].absorb_df
    deriv_df = ana.derivative(df)*10
    plot.mult_func([260], [df - df.min(axis=1).loc[260], deriv_df], subtitle=[key], linestyle=["-",":"], marker=["x",""])

In [ ]:
for key in

### Linear LM Fit for Melting Temperature


<div class="alert alert-block alert-info">
<b>Info:</b> Determination of melting temperatures and ssDNA fraction
</div>

In [ ]:
print(data_all.keys())

In [15]:

melt_T = {}
comp2_conc = {}
#liste = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M', 'GdmCl_6M',  'GdmSCN_0.5M', 'Urea_2M']
#liste = ['0 M Control 10 mM Mg', '05 M Gdm2SO4 100 mM Mg', '05 M Gdm2SO4 400 mM Mg', '1 M GdmCl 100 mM Mg', '1 M GdmCl 400 mM Mg', '1 M GdmCl 400 mM Mg 2', '2 M Gdm2SO4 100 mM Mg', '2 M Gdm2SO4 400 mM Mg', '4 M GdmCl', '4 M GdmCl 100 mM Mg', '4 M GdmCl 100 mM Mg 2', '4 M GdmCl 400 mM Mg']
liste = ['0 M Control 10 mM Mg']
#liste = data_all.keys()

In [21]:
for key in liste:

    # fit three secitons
    df =  data_all[key].absorb_df.loc[:,:80]
    #high_fit, high_params, high_std =  ana.lm_fit(df.iloc[:,-3:], f_type='linear')
    high_fit, high_params, high_std =  ana.lm_fit(df.iloc[:,-5:], f_type='linear')
    #low_fit, low_params, low_std =  ana.lm_fit(df.iloc[:,:3], f_type='linear')
    low_fit, low_params, low_std =  ana.lm_fit(df.iloc[:,:10], f_type='linear')


    # normalize absorbance values and get next 3 values to 0.5
    df_norm = ana.normalize(df).loc[260, :]
    df_sort = pd.DataFrame(df_norm.iloc[(df_norm-0.5).abs().argsort()[:3]], columns=[260])
    # make linear fit
    middle_fit, middle_params, middle_std = ana.lm_fit(df.loc[:, df_sort.index], f_type='linear')


    # calculate cross of middle and median
    melt_T[key] = ( ( (- high_params["y0"] - low_params["y0"]) / 2 ) + middle_params["y0"] ) / ( ( (high_params["m"] + low_params["m"]) / 2 ) - middle_params["m"] ) 
    # calculate error 
    teiler = ( ( (high_params["m"] + low_params["m"]) / 2 ) - middle_params["m"] ) 
    nenner =  ( ( (- high_params["y0"] - low_params["y0"]) / 2 ) + middle_params["y0"] ) 
    t_melt_error =  np.sqrt( (-0.5 * high_std["y0"] / teiler)**2 + (-0.5 * low_std["y0"] / teiler)**2 + (middle_std["y0"] / teiler)**2
                              + (nenner * 0.5 * high_std["m"] / (teiler**2) )**2 + (nenner * 0.5 * low_std["m"] / (teiler**2) )**2 + (nenner * -1 * middle_std["m"] / (teiler**2) )**2)

    
    # make nice plot
    plot_fit= pd.DataFrame(ana.linear(df.columns, *high_params.values()), columns=["high plateu"], index=df.columns)
    plot_fit["low plateu"] = ana.linear(df.columns, *low_params.values())
    plot_fit["middle"] = ana.linear(df.columns, *middle_params.values())
    plot_fit["Median"] = 0.5*(plot_fit["high plateu"] + plot_fit["low plateu"])
    #ssDNA
    comp2_conc[key] = (df.loc[260] - plot_fit["low plateu"]) / (plot_fit["high plateu"] - plot_fit["low plateu"])

    plot.mult_func([260, "high plateu", "low plateu", "middle", "Median"], [df, plot_fit.T], 
                   vertical_line=[melt_T[key]], marker=["X", "", "", "", ""], linestyle=["--", ":", ":", "-.", "-."],
                   y_scaling=True, y_min=[df.min(axis=1).loc[260] - 0.02 ], y_max=[df.max(axis=1).loc[260] + 0.02], 
                   title="", subtitle=[key], sec_ax=[df.min(axis=1).loc[260], df.max(axis=1).loc[260]], y_label="absorption / %",
                   x_label='temperature / °C', label=["total absorption",  "upper baseline", "lower baseline", "linear interpolation", "center line", "melting temperature"])
    
    print(melt_T)
    print(comp2_conc[key])

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5
    # variables        = 2
    chi-square         = 2.9752e-07
    reduced chi-square = 9.9174e-08
    Akaike info crit   = -79.1860846
    Bayesian info crit = -79.9672088
[[Variables]]
    m:   7.6820e-04 +/- 9.9586e-05 (12.96%) (init = 1)
    y0:  0.45460540 +/- 0.00776900 (1.71%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 2.5465e-07
    reduced chi-square = 3.1831e-08
    Akaike info crit   = -170.859561
    Bayesian info crit = -170.254391
[[Variables]]
    m:   1.9850e-04 +/- 1.9643e-05 (9.90%) (init = 1)
    y0:  0.38886893 +/- 1.2188e-04 (0.03%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(m,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'0 M Control 10 mM Mg': 65.33143041589959}
1     0.004186
2     0.000689
3    -0.000570
4    -0.002690
5    -0.002077
6    -0.003499
7     0.000696
8     0.000541
9    -0.000585
10    0.003318
11    0.007285
12    0.008364
13    0.008648
14    0.010772
15    0.011113
16    0.018009
17    0.019179
18    0.027372
19    0.030845
20    0.038260
21    0.049556
22    0.050696
23    0.062919
24    0.066000
25    0.078289
26    0.081094
27    0.094867
28    0.097297
29    0.107692
30    0.121771
31    0.126497
32    0.136673
33    0.141365
34    0.150271
35    0.156491
36    0.165631
37    0.176011
38    0.182491
39    0.188909
40    0.191845
41    0.203464
42    0.204106
43    0.212188
44    0.214672
45    0.220946
46    0.227750
47    0.230461
48    0.235190
49    0.238153
50    0.244445
51    0.245028
52    0.250186
53    0.253480
54    0.260745
55    0.269140
56    0.270965
57    0.277941
58    0.282509
59    0.289812
60    0.294570
61    0.303244
62    0.312345
63    0.336399
64    0.359

In [ ]:
print(melt_T)

In [ ]:
x = [10, 100, 400]
y =  [melt_T['4_M_GdmCl_10_mM_Mg'], melt_T['4_M_GdmCl_100_mM_Mg'], melt_T['4_M_GdmCl_400_mM_Mg']]

graph1 = pd.DataFrame(y, index=x, columns=["Mg + 4 M GdmCl"]).T


plot.mult_func(["Mg + 4 M GdmCl"], [graph1], title="", subtitle=[""], 
               x_label="Mg+ concentration / mM", y_label="temperature / °C", label=[ r"$GdmCl$ "],
               marker=["X","o"], linestyle=["-.", "-",], backgroundcolor='white', colors=['red',  'black'])

In [ ]:
x = [0., 0.5, 1., 2., 4., 6.]
y =  [melt_T["Control_0M"], melt_T["Gdm2SO4_0.5M"], melt_T["Gdm2SO4_1M"], melt_T["Gdm2SO4_2M"], melt_T["Gdm2SO4_4M"], melt_T["Gdm2SO4_6M"]]

graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_1M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"], melt_T["GdmCl_6M"]]

graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="", subtitle=[""], 
               x_label="salt concentration / M", y_label="temperature / °C", label=[r'$Gdm_{2}SO_{4}$',  r"$GdmCl$ "],
               marker=["X","o"], linestyle=["-.", "-",], backgroundcolor='white', colors=['red',  'black'], error={"Gdm2SO4": 0.5, "GdmCl": 0.5})

## ITTFA fractions.txt generator

<div class="alert alert-block alert-info">
<b>Info:</b> Creates the fractions text files.txt - files which are used for ITTFA.
    Melting Temperature must before be determined to calculate ssDNA fraction
</div>

In [ ]:
# folder path to load spectra and to save fraction profiles
name = os.path.join(ITTFA_path, 'fractions.txt')
key = '0 M Control 10 mM Mg'
#ITTFA_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Factor_analysis\\ITTFA"

# print key folder to get the name structure 
print(data_all.keys())

In [ ]:
name = os.path.join(ITTFA_path, 'fractions.txt')
key = 'Control_0M'

In [ ]:
# create file
f = open(name,"w+")

# extra spectra
extra_spectra = 0
components = 3

# write in file
for k in range(components):
    for i in range((len(data_all[key].cd_df.loc[:, 20:90].columns) + extra_spectra)):
        if k == 1 and i < len(data_all[key].cd_df.loc[:, 20:90].columns):
            f.write("\t" + str(k+1) + "\t" + str(i+1) + "\t" + str(abs(comp2_conc[key].iloc[i].round(2))) + " \t 1 \n")
        elif i >= len(data_all[key].cd_df.loc[:, 20:90].columns) and  k == (i % len(data_all[key].cd_df.loc[:, 20:90].columns)):
            f.write("\t" + str(k+1) + "\t" + str(i+1) + "\t" + str(1) + "\t 1 \n")
        else:
            f.write("\t" + str(k+1) + "\t" + str(i+1) + "\t 0 \t 0 \n")

# close file
f.close()
# Bei fehler: "name 'comp2_conc' is not defined" den LM Fit laufen lassen

In [ ]:
print(data_all[key].t_list)

## Derivative of Absorbtion


In [26]:
for key in data_all.keys():
    deriv =  data_all[key].absorb_df.diff(axis = 1) / 0.5
    data = data_all[key].absorb_df
    plot.mult_func([260], [data], [deriv], title= key, subtitle=["Absolute value", "interpolated derivative"], label=[""], y_label="Absorbance")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plot.mult_func([260], [data], [deriv], title= "Hypochromatic shift", subtitle=["Absolute value", "interpolated derivative"], label=[""], y_label="Absorbance")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Calibration 

In [ ]:
datapath = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Data\\Calibration"
datalist = os.listdir(datapath)
# 0 for PL, 1 for CSA
filename = os.path.join(datapath, datalist[0])

print(filename)

In [ ]:
file = open(filename, "r")
list_of_lines = file.readlines()

# Parameter for head and tail part of document which will be deleted
head = 21
tail = 1572
del list_of_lines[tail:]
del list_of_lines[:head]

# separating numbers into sublist
for i in range(len(list_of_lines)):
    list_of_lines[i] = list_of_lines[i].replace("\n", "")
    list_of_lines[i] = list_of_lines[i].replace(",", ".")
    list_of_lines[i] = list_of_lines[i].split('\t', 3)

# transform list of lists to array of float-type
matrix = np.array(list_of_lines)
matrix = matrix.astype(np.float)
file.close()

calibration_data = pd.DataFrame(matrix, columns=['wavelength', 'CD Values', 'HT Values', 'Absorbance'])
calibration_data = calibration_data.set_index('wavelength')

In [ ]:
plot.mult_func(['CD Values'], [calibration_data], swap=True)
plot.mult_func(['Absorbance'], [calibration_data], swap=True)
plot.mult_func(['HT Values'], [calibration_data], swap=True)

**Molar Ellipticity**

In [ ]:
# for PL
molarity = calibration_data.loc[215,'Absorbance']  * 10**4 / 90 # molarity in mMs
print('Determined concentration in mmol/l: \t', molarity)
pathlength =  0.1 # cm 
print('Ratio experimental vs. theoretical value at the maximum around 290 nm: \n with determinded molarity: ',
      calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* molarity) / (-4.9*3300),
      '\t  with wanted molarity 11.53 mM:' , calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* 11.53) / (-4.9*3300) ) 
      

In [ ]:
# for CSA
molarity = calibration_data.loc[285,'Absorbance'] *  (10**4) / 34.5  # molarity in mMs
print('Determined concentration in mmol/l: \t', molarity)
pathlength =  0.1 # cm 
print('Ratio experimental vs. theoretical value at the maximum around 290 nm: \n with determinded molarity: ',
      calibration_data.loc[calibration_data['CD Values'].idxmax(axis=0), 'CD Values']  / (0.001* molarity) / (7800),  
      '\t  with wanted molarity 4.7 mM:' , calibration_data.loc[calibration_data['CD Values'].idxmax(axis=0), 'CD Values']  / (0.001* 4.7) / (7800)  ) 
print('Ratio experimental vs. theoretical value at the minimum around 192.5 nm: \n with determinded molarity: ',
      calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* molarity) / (-15600 ),  
      '\t  with wanted molarity 4.7 mM:' , calibration_data.loc[calibration_data['CD Values'].idxmin(axis=0), 'CD Values']  / (0.001* 4.7) / (-15600 ) )
print('Ratio between absorbance 285 nm and molar ellipticity at 290.5 nm: ', 
      (calibration_data.loc[calibration_data['CD Values'].idxmax(axis=0), 'CD Values']  / (3300)) / calibration_data.loc[285, 'Absorbance'],
     ' -> this value should be 0.068') # should be 0.068
 

# Test

**Zero Crossing**

In [ ]:
print(data_all.keys())

In [ ]:
zero_crossing = {}

wave_max = 280
wave_min = 240
for key in data_all.keys():
    df = data_all[key].cd_df.loc[wave_min:wave_max]
    zeros = pd.DataFrame()
    for col in df.columns:
        sgn = np.sign(df.loc[wave_min, col])
        zero_liste = []
        for idx in df.index:
            if np.sign(df.loc[idx, col]) != np.sign(sgn):
                zero_liste.append(int(idx)+0.5)
            sgn = np.sign(df.loc[idx, col])
        new = pd.DataFrame({col: zero_liste})
        zeros = pd.concat([zeros, new], axis=1)
    zero_crossing[key] = zeros

In [ ]:
for key in data_all.keys():
    plot.mult_func([0], [zero_crossing[key]], subtitle=[key])

**Derivative**

In [ ]:
keys = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'Gdm2SO4_4M - Wiederholung', 'Gdm2SO4_6M', 'GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_2M_20nM_Origami', 'GdmCl_2M_24h', 'GdmCl_2M_80nM_Origami', 'GdmCl_4M', 'GdmCl_6M', 'GdmCl_6M_70', 'GdmSCN_0.5M', 'GdmSCN_2M', 'GdmSCN_4M', 'Urea_05M', 'Urea_2M']
#keys = ['1_M_GdmCl_100_mM_Mg',  '4_M_GdmCl_100_mM_Mg', '4_M_GdmCl_10_mM_Mg', '4_M_GdmCl_400_mM_Mg']

In [ ]:
for key in keys:
    df = data_all[key].cd_df
    data = df.to_numpy()
    t_vec = df.columns.to_numpy()
    delta_x = data[:, 2:] - data[:, :-2]
    delta_t = t_vec[2:] - t_vec[:-2]
    deriv =  pd.DataFrame(np.divide(delta_x, delta_t), index=df.index, columns=df.iloc[:,1:-1].columns)
    deriv = deriv.astype('float64')
    plot.heatmap(deriv.loc[215:300], subtitle=[key], title='')

**Concentration calculation**

In [ ]:
df = pd.DataFrame()
liste = ['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M', 'Gdm2SO4_4M','GdmCl_0.5M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M']
for key in liste:
    df[key] = data_all[key].cd_df[70]
    
plot.mult_func(liste, [df], swap=True, x_min=215, x_max=300, subtitle=['Data at 70°C'], x_label='Wavelengt / nm' )

In [ ]:
for key in data_all.keys():
    absorb_coeff = 0.0263 #ml/µg
    conc = data_all[key].absorb_df.loc[260,20] / (absorb_coeff* 0.1) # µg/ml
    mol_weight = 4472760.4 *(10**(6)) # µg/mol
    conc = conc / mol_weight #mol/ml
    conc =  conc * (10**(3)) # mol/L = M
    conc = conc * (10**(9)) # nmol/L = nM
    
    conc2 = data_all[key].absorb_df.loc[260,20] / (0.02* 0.1) # µg/ml
    mol_weight = 4472760.4 *(10**(6)) # µg/mol
    conc2 = conc2 / mol_weight #mol/ml
    conc2 =  conc2 * (10**(3)) # mol/L = M
    conc2 = conc2 * (10**(9)) # nmol/L = nM

    print(key, round(conc,3), round(conc2, 3), round(conc-conc2,3 ))

In [ ]:
#Daniels path:
print_path = "F:\\HZDR\\CD_Auswertung\\c"
#Chrisophs path
#print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Auswertung"

In [ ]:
excel_path = print_path + "\\" + "\\output.xlsx"
print(excel_path)

In [ ]:
liste = ['Control_0M', 'GdmCl_1M', 'GdmCl_2M', 'GdmCl_4M', 'Gdm2SO4_0.5M', 'Gdm2SO4_1M', 'Gdm2SO4_2M']
for key in liste:
    df = data_all[key].absorb_df.loc[260]
    with pd.ExcelWriter(excel_path, mode='a') as writer:  
        df.round(3).to_excel(writer, sheet_name=key)

**Simulation of Melting Curves**

In [ ]:
# gas constant
r =  8.314

# first process -> melting dsDNA to ssDNA in tensed state
t_01 =  337
dh_1 = 123000

# second process -> melting dsDNA to ssDNA in relaxed state
t_02 = 335.9375
dh_2 = 215000

# third process -> relaxing origami in dsDNA
t_03 = 323.14502
dh_3 = 10000 

# add dc_p values
dc_1 = 2000
dc_2 = 4000
dc_3 = 1000

# temperature points and dt to reference temperature t_01
t =  np.arange(273, 372, 1)

# create lists to save calcua
rel_conc = pd.DataFrame(index=t, columns=[1,2,3,4])

In [ ]:
# calculate fractions for all temperature points
for i in t:
    # calculate dg
    dg_1 = - (dh_1/t_01 - dc_1) * (i - t_01) + dc_1 * i * np.log(t_01/i) # dsDNA to ssDNA in tensed state 
    dg_2 = - (dh_2/t_01 - dc_2) * (i - t_02) + dc_2 * i * np.log(t_02/i) # dsDNA to ssDNA in relaxed state
    dg_3 = - (dh_3/t_01 - dc_3) * (i - t_03) + dc_3 * i * np.log(t_03/i) # tensed state to relaxed state in dsDNA
    
    # calculate equilibrium constants k
    k_1 = np.exp( - dg_1 / (r * i))
    k_2 = np.exp( - dg_2 / (r * i))
    k_3 = np.exp( - dg_3 / (r * i))
    
    # calculate relative concentrations
    sum_of_states = 1 + k_1 + k_3 + k_3 * k_2 # number of states / number of state 1 
    rel_conc.loc[i, 1] = 1 / sum_of_states # norm.population of dsDNA tensed 
    rel_conc.loc[i, 2] = k_1 /sum_of_states # norm.population of ssDNA tensed
    rel_conc.loc[i, 3] = k_3 /sum_of_states # norm.population of dsDNA relaxed
    rel_conc.loc[i, 4] = k_3 * k_2 /sum_of_states # norm.population of ssDNA relaxed 


In [ ]:
plot.mult_func([1,2,3,4], [rel_conc.T])

**Beers Law**

In [ ]:
for key in data_all.keys():
    absorbance = data_all[key].absorb_df.loc[260, 20]
    df = data_all[key].cd_df
    
    #conc_ds =  absorbance/(0.02*0.1) #[µg/mL]
    conc_ds =  absorbance/(0.0263*0.1) #[µg/mL] ecact for gc content in origami triangles
    molar = conc_ds * 10**(-6) / (4472760.4 * 10**(-3)) * (10**9) #[nmol/l] 
    df_norm = 100 * df / (molar * 0.1)    
    print(key, ":  \t", molar , 'nM')

In [ ]:
for key in data_all.keys():
    plot.mult_func([280], [data_all[key].cd_df], subtitle=[key], y_scaling=False, y_max=[], y_min=[])

In [ ]:


        # put second y_axis for "GdmCl_0.5M" normed
        # function to bring back values
        def expand_back(y):
            return y * (0.523893 - 0.379531) + 0.379531
        # inverse function
        def norm(y):
            return (y - 0.379531) / (0.523893 - 0.379531)

        # put axis and label
        secax = ax.secondary_yaxis('right', functions=(expand_back, norm), fontsize=16)
        secax.set_ylabel('Absorption [%]', fontsize=16)

In [ ]:
df = data_all["Control_0M"].cd_df
mean = df.mean(axis=1)
var =  df.var(axis=1)
dyn = df.subtract(mean, axis=0)

col = dyn.columns 
idx = dyn.index
m = len(col)
n =len(idx)

In [ ]:
wave1 =  247
wave2 = 260
i = 20

print(np.sum((dyn.loc[wave1, :]/mean[wave1] - dyn.loc[wave2, :]/mean[wave2]) * col) * np.sqrt(var.loc[wave1]*var.loc[wave2])/(m*(m-1)) )


In [ ]:

assync = pd.DataFrame(np.zeros((n,n)), index=idx, columns=idx)

for j in idx:
    for k in idx:
        assync.loc[j,k] = np.sum((dyn.loc[j, :]/mean[j] - dyn.loc[k, :]/mean[k]) * col) * np.sqrt(var.loc[j]*var.loc[k])/(m*(m-1))
print(assync)

In [ ]:
plot.heatmap(assync, x_min=[220], y_min=[220])

In [ ]:
# read by default 1st sheet of an excel file
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Projekt\\Auswertung\\"
data_path1 = os.path.join(print_path, 'AveragedSpectra.xlsx')
df= pd.read_excel(data_path1)
df= df.drop(0, axis=0)
df = df.set_index('Wavelength')
plot.mult_func(['S1', 'S2', 'S2\'', 'S3'], [df.T], marker=['','','',''], subtitle=[''], x_label='wavelength / nm',
              colors=['tab:red', 'tab:blue', 'tab:grey', 'tab:green'], y_label='relative ellipticity', baseline=True, backgroundcolor='#FAFFCD')

# MD H-bonds Auswertung

In [ ]:
print_path = "C:\\Users\\crazy\\Documents\\Uni\\Masterarbeit\\Thesis\\MD"

In [ ]:
liste = ['Hbonds-Gdm2SO4-DNA_to_Gdm.dat', 'Hbonds-Gdm2SO4-DNA_to_wat.dat','Hbonds-Gdm2SO4-withinDNA.dat',
         'Hbonds-GdmCl-DNA_to_Gdm.dat', 'Hbonds-GdmCl-DNA_to_wat.dat', 'Hbonds-GdmCl-withinDNA.dat']

In [ ]:
for name in liste:

    
    # get data path
    data_path = os.path.join(print_path, name)
  
    # open file and create data list
    file = open(data_path, "r")
    data = []
        
    # list of lines, of which all elemnts equal to space or enter are used as split, and then empty elements are deleted
    list_of_lines = file.readlines()
    
    
    for k in range(len(list_of_lines)):
        data.extend(re.split(' |\n', list_of_lines[k]))
        data1 = np.array(list(filter(lambda x: x != '', data)))
    
  
    # data list reshaped as 2D array and added to plot
    data1 = data1.reshape(len(list_of_lines), 2)

    df =  pd.DataFrame(data1, columns=['frame', 'Hbonds'])
    df = df.set_index('frame')
    df = df.astype('int')
    print(name, ': \ ', df['Hbonds'].mean(), '+-', df['Hbonds'].std())
    
    #array = np.array(df['Hbonds'].min(), df['Hbonds'].max()+1, 2)
    
    # setup the plot
    
    fig,ax = plt.subplots(1,1)
    
    df.hist(column='Hbonds', bins=np.arange(50, 101, 2), grid=False, histtype='stepfilled', ax=ax)
    ax.set_title(name)
    ax.set_xlabel('number of h-bonds')
    ax.set_ylabel('number of frames')
    plt.show()

# Parafak Ethanol

## *setting up the Notebook*

In [ ]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib

from scipy import integrate
import lmfit
import os

In [ ]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [ ]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [ ]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [ ]:
# Path for Daniel:
path = "F:\\HZDR\\CD_data"

# My Path:
#path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\Ethanol"
datalist = os.listdir(path)
print(datalist)

In [ ]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [ ]:
df =  data_all["Control_0%"].cd_df
df = df.rename(columns={-1:"0%"})
df["50%"] =  (data_all["Ethanol_50%"].cd_df.loc[:,-1])
df["nach"] =  (data_all["Ethanol_50%_nachTest"].cd_df.loc[:,-1])
df["vor"] =  (data_all["Ethanol_50%_vorTest"].cd_df.loc[:,-1])
df["50% real"] =  (data_all["Ethanol_50%_real"].cd_df.loc[:,-1])
print(df)

In [ ]:
plot.mult_func(['0%', 'vor', 'nach', '50%', "50% real" ], [df], swap=True, marker=["", "", "", "", ""], x_label="Wavelengt [nm]")

# Salmon testes

## setting up the notebook


In [ ]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
import openpyxl

from scipy import integrate
import lmfit
import os

In [ ]:
import analise as ana
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [ ]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [ ]:
# change plot size
plt.rcParams["figure.figsize"] = (10,7)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [ ]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\Salmon Testes"
datalist_salmon = os.listdir(path)
print(datalist_salmon)

In [ ]:
# all data in one dictionary with foldernames as names
data_salmon = {}
for i in range(len(datalist_salmon)):
    data_salmon[datalist_salmon[i]] = cdata.CData(os.path.join(path, datalist_salmon[i]))

In [ ]:
for key in data_salmon.keys():
    absorbance = data_salmon[key].absorb_df.loc[260, 20]
    df = data_salmon[key].cd_df
    
    conc_ds =  absorbance/(0.02*0.1*(10**3)) #[mg/mL]
    print(key, ":  \t", conc_ds)

In [ ]:
for key in data_salmon.keys():
    plot.mult_func([260], [data_salmon[key].absorb_df], subtitle=[key])

In [ ]:
print(data_salmon["Control_0M_continous"].t_list)

In [ ]:
plot.mult_func([20, 40, 60, 80], [data_salmon['Control_0M_continous'].cd_df], swap=True, baseline=True, marker=["","","",""], subtitle=["Salmon Control"], linestyle=["-",":", ":", "-"], x_min=220)